##### Trend Template
1. The current stock price is above both the 150-day (30-week) and the 200-day (40-week) moving average price lines.
2. The 150-day moving average is above the 200-day moving average.
3. The 200-day moving average line is trending up for at least 1 month (preferably 4–5 months minimum in most cases).
4. The 50-day (10-week) moving average is above both the 150-day and 200-day moving averages.
5. The current stock price is trading above the 50-day moving average.
6. The current stock price is at least 30 percent above its 52-week low. (Many of the best selections will be 100 percent, 300 percent, or greater above their 52-week low before they emerge from a solid consolidation period and mount a large scale advance.)
7. The current stock price is within at least 25 percent of its 52-week high (the closer to a new high the better).
8. The relative strength ranking (as reported in Investor’s Business Daily) is no less than 70, and preferably in the 80s or 90s, which will generally be the case with the better selections.

In [1]:
import baostock as bs
import pandas as pd
import talib
import time
import random
from tqdm import tqdm

In [2]:
START_DATE = '2010-01-01'

In [3]:
bs.login()

login success!


In [4]:
# get all stock code
rs = bs.query_stock_basic()
data_list = []
while rs.error_code == '0' and rs.next():
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
codes = result.query('type=="1" and status=="1"')['code']
print('Get', len(codes), 'stocks.')

Get 4072 stocks.


In [5]:
candidate = []
for i, code in enumerate(tqdm(codes)):
    rs = bs.query_history_k_data_plus(code, "date,close", start_date=START_DATE, frequency='d', adjustflag='2')
    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    result = pd.DataFrame(data_list, columns=rs.fields)
    close = result['close'].astype('float64')
    sma50 = talib.SMA(close, 50)
    sma150 = talib.SMA(close, 150)
    sma200 = talib.SMA(close, 200)
    delta = 0
    while True:
        if delta + 52*5 >= len(close):
            break
        close52 = close[-delta-52*5:][:52*5]
        close52low = close52.min()
        close52high = close52.max()
        match = False
        if close.iloc[-delta-1] > sma50.iloc[-delta-1] > sma150.iloc[-delta-1] > sma200.iloc[-delta-1]:
            if (sma200.pct_change()[-delta-20:][:20] > 0).all():
                if close.iloc[-delta-1] > close52low * 1.30 and close.iloc[-delta-1] > close52high * 0.75:
                    delta += 1
                    match = True
        if not match:
            break
            
    if delta > 0:
        candidate.append((code, delta, close.iloc[-1], sma50.iloc[-1], sma150.iloc[-1], sma200.iloc[-1], close52low, close52high))
    
    if i % 100 == 0:
        time.sleep(random.random())
print('Get', len(candidate), 'candidates.')
result = pd.DataFrame(candidate, columns=('code', 'delta', 'close', 'sma50', 'sma150', 'sma200', '52low', '52high'))
result.sort_values(['delta'])

100%|██████████| 4072/4072 [07:07<00:00,  9.53it/s]

Get 630 candidates.


,code,delta,close,sma50,sma150,sma200,52low,52high
295,sz.000789,1,15.73,15.5748,15.096507,14.010824,8.964818,19.600000
600,sz.300664,1,9.92,9.8560,9.512078,9.464160,7.238214,11.240000
250,sz.000039,1,14.68,10.5498,8.717532,8.574621,6.778568,16.280000
484,sz.002926,1,12.19,12.0858,11.598795,11.453604,9.200403,14.188094
59,sh.600552,1,6.62,6.4802,6.096862,6.032206,4.919421,7.699535
...,...,...,...,...,...,...,...,...
474,sz.002884,132,32.06,25.3184,19.192217,17.905620,10.567084,16.056587
12,sh.600132,139,103.20,100.8588,76.271657,68.853873,32.875399,57.028984
90,sh.600809,153,273.21,213.4528,163.313066,144.537499,50.317363,99.494153
111,sh.601100,159,87.86,74.9674,60.447539,54.361312,17.920737,41.508531


In [ ]:
bs.logout()

In [59]:
rs = bs.query_history_k_data_plus('sh.600006', "date,close", start_date=START_DATE, frequency='d', adjustflag='2')
data_list = []
while (rs.error_code == '0') & rs.next():
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)

In [60]:
close = result['close']
sma200 = talib.SMA(close, 200)

In [64]:
len(close[-52-10:][:52])

52

In [13]:
result['code'] = result['code'].apply(lambda x: x[3:])
result.sort_values(['delta']).to_excel('output.xlsx')
# result.sort_values(['delta']).to_excel(writer)
# writer.save()

In [12]:
result

,code,delta,close,sma50,sma150,sma200,52low,52high
0,sh.600006,22,6.25,5.3830,4.590541,4.463907,3.629222,5.360000
1,sh.600031,93,28.30,25.4224,21.715464,20.427199,11.503178,20.182251
2,sh.600036,13,43.00,39.2724,36.103107,35.437659,28.428707,41.220000
3,sh.600039,12,5.22,4.5458,4.214233,4.036168,2.978769,4.610000
4,sh.600059,10,10.44,9.3272,9.176175,8.792971,6.983759,11.340000
...,...,...,...,...,...,...,...,...
625,sz.300758,2,29.29,27.0048,23.143299,21.466475,14.500343,36.540000
626,sz.300759,28,115.75,107.1412,92.036696,83.739420,41.259838,116.160000
627,sz.300762,6,46.22,40.9868,37.817221,36.655875,28.287022,49.950000
628,sz.300769,27,146.00,99.1172,93.886530,92.536151,43.532193,122.380000


In [11]:
result['code'].apply(lambda x: x[3:])

0      600006
1      600031
2      600036
3      600039
4      600059
        ...  
625    300758
626    300759
627    300762
628    300769
629    300782
Name: code, Length: 630, dtype: object